In [1]:
import sys
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
from sklearn.cluster import KMeans # type: ignore
from sklearn.metrics.pairwise import euclidean_distances # type: ignore
from sklearn.linear_model import LinearRegression # type: ignore
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
sys.path.append(os.path.abspath('C:/Users/nejat/AIM Projects/Telecommunication Data Analysis/src'))
from db_connection import PostgresConnection
from data_cleaning import preprocess_data  
import psycopg2
from psycopg2.extras import execute_values

engagement_df = pd.read_csv('user_engagement_results.csv')
experience_df = pd.read_csv('experience_analysis_results.csv')

print(engagement_df.head())
print(experience_df.head())

   MSISDN/Number  Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  Start  \
0   3.360100e+10   116720.0      8.426375e+08        36053108.0      1   
1   3.360100e+10   181230.0      1.207552e+08        36104459.0      1   
2   3.360100e+10   134969.0      5.566597e+08        39306820.0      1   
3   3.360101e+10    49878.0      4.019932e+08        20327526.0      1   
4   3.360101e+10    37104.0      1.363130e+09        94280527.0      2   

   cluster  
0        0  
1        0  
2        0  
3        0  
4        2  
   MSISDN/Number  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
0   3.360100e+10                 2.080991e+07        46.000000   
1   3.360100e+10                 2.080991e+07        30.000000   
2   3.360100e+10                 2.080991e+07       109.795706   
3   3.360101e+10                 1.066000e+03        69.000000   
4   3.360101e+10                 1.507977e+07        57.000000   

   Avg Bearer TP DL (kbps)                    Handset Type  cluster  
0         

In [2]:
def calculate_engagement_experience_scores(engagement_df, experience_df):
    combined_df = pd.merge(engagement_df, experience_df, on='MSISDN/Number')
    
    kmeans_engagement = KMeans(n_clusters=3, random_state=42)
    engagement_clusters = kmeans_engagement.fit_predict(combined_df[['Dur. (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)']])
    least_engaged_centroid = kmeans_engagement.cluster_centers_[0] 

    combined_df['engagement_score'] = euclidean_distances(combined_df[['Dur. (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)']], 
                                                         [least_engaged_centroid]).flatten()

    kmeans_experience = KMeans(n_clusters=3, random_state=42)
    experience_clusters = kmeans_experience.fit_predict(combined_df[['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']])
    worst_experience_centroid = kmeans_experience.cluster_centers_[0]  

    combined_df['experience_score'] = euclidean_distances(combined_df[['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']], 
                                                           [worst_experience_centroid]).flatten()
    combined_df['satisfaction_score'] = (combined_df['engagement_score'] + combined_df['experience_score']) / 2
    return combined_df


combined_df = calculate_engagement_experience_scores(engagement_df, experience_df)

top_10_satisfied_customers = combined_df[['MSISDN/Number', 'satisfaction_score']].sort_values(by='satisfaction_score').head(10)
print(top_10_satisfied_customers)

C:\Users\nejat\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\nejat\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


        MSISDN/Number  satisfaction_score
30153    3.365825e+10        2.380003e+06
42669    3.366106e+10        3.454811e+06
67560    3.366795e+10        4.755155e+06
49573    3.366268e+10        4.755995e+06
67561    3.366795e+10        4.908502e+06
100494   3.376388e+10        5.522642e+06
46237    3.366191e+10        6.153185e+06
59908    3.366541e+10        6.256232e+06
41908    3.366086e+10        6.385396e+06
88677    3.369947e+10        6.416834e+06


In [3]:
X = combined_df[['engagement_score', 'experience_score']]
y = combined_df['satisfaction_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error of the model: {mse:.4f}")

Mean Squared Error of the model: 0.0000


In [4]:
features = combined_df[['engagement_score', 'experience_score']]

kmeans = KMeans(n_clusters=2, random_state=42)
combined_df['experience_engagement_cluster'] = kmeans.fit_predict(features)

print(combined_df.head())

C:\Users\nejat\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


   MSISDN/Number  Dur. (ms)  Total DL (Bytes)  Total UL (Bytes)  Start  \
0   3.360100e+10   116720.0      8.426375e+08        36053108.0      1   
1   3.360100e+10   181230.0      1.207552e+08        36104459.0      1   
2   3.360100e+10   134969.0      5.566597e+08        39306820.0      1   
3   3.360101e+10    49878.0      4.019932e+08        20327526.0      1   
4   3.360101e+10    37104.0      1.363130e+09        94280527.0      2   

   cluster_x  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
0          0                 2.080991e+07        46.000000   
1          0                 2.080991e+07        30.000000   
2          0                 2.080991e+07       109.795706   
3          0                 1.066000e+03        69.000000   
4          2                 1.507977e+07        57.000000   

   Avg Bearer TP DL (kbps)                    Handset Type  cluster_y  \
0                     37.0  Huawei P20 Lite Huawei Nova 3E          0   
1                     48.0          

In [5]:
print(combined_df.columns)

Index(['MSISDN/Number', 'Dur. (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)',
       'Start', 'cluster_x', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)',
       'Avg Bearer TP DL (kbps)', 'Handset Type', 'cluster_y',
       'engagement_score', 'experience_score', 'satisfaction_score',
       'experience_engagement_cluster'],
      dtype='object')


In [6]:
cluster_summary = combined_df.groupby('experience_engagement_cluster')[['satisfaction_score', 'experience_score']].mean()
print("Cluster Summary:\n", cluster_summary)

Cluster Summary:
                                satisfaction_score  experience_score
experience_engagement_cluster                                      
0                                    6.857367e+08      1.313037e+07
1                                    3.825166e+08      1.239207e+07


In [ ]:
def export_to_postgres(df, table_name):
    connection = psycopg2.connect(
        host="localhost",
        user="postgres",
        password="237132",
        database="tellco"
    )
    cursor = connection.cursor()

    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        msisdn_number VARCHAR(255),
        engagement_score FLOAT,
        experience_score FLOAT,
        satisfaction_score FLOAT,
        experience_engagement_cluster INT
    );
    """)
    
    required_columns = ['MSISDN/Number', 'engagement_score', 'experience_score', 'satisfaction_score', 
                         'experience_engagement_cluster']
    
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"Missing columns in DataFrame: {missing_columns}")
        return

    data = [
        (
            row['MSISDN/Number'], 
            row['engagement_score'], 
            row['experience_score'], 
            row['satisfaction_score'],
            row['experience_engagement_cluster']
        )
        for _, row in df.iterrows()
    ]

    query = f"""
    INSERT INTO {table_name} (
        msisdn_number, engagement_score, experience_score, satisfaction_score, experience_engagement_cluster
    ) VALUES %s;
    """
    
    execute_values(cursor, query, data)

    connection.commit()
    cursor.close()
    connection.close()

export_to_postgres(combined_df, 'user_satisfaction_data')

In [11]:
def query_from_postgres():
    connection = psycopg2.connect(
        host="localhost",
        user="postgres",
        password="237132",
        database="tellco"
    )
    query = "SELECT * FROM user_satisfaction_data LIMIT 5;"
    df = pd.read_sql(query, connection)
    connection.close()
    return df

output_df = query_from_postgres()
print("Sample data from PostgreSQL:\n", output_df)

Sample data from PostgreSQL:
    msisdn_number  engagement_score  experience_score  satisfaction_score  \
0  33601001722.0      8.730792e+08      1.700644e+07        4.450428e+08   
1  33601001754.0      1.592802e+09      1.700644e+07        8.049041e+08   
2  33601002511.0      1.157630e+09      1.700644e+07        5.873182e+08   
3  33601007832.0      1.313310e+09      3.802498e+06        6.585564e+08   
4  33601008617.0      3.493708e+08      1.127628e+07        1.803235e+08   

  engagement_cluster experience_cluster  experience_engagement_cluster  
0               None               None                              1  
1               None               None                              0  
2               None               None                              0  
3               None               None                              0  
4               None               None                              1  


C:\Users\nejat\AppData\Local\Temp\ipykernel_20236\1226917414.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
